In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=.05))
def model(x,w,w2,w3,w4,w_o,p_keep_conv,p_keep_hid):
    l1a = tf.nn.relu(tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME'))
    l1 = tf.nn.max_pool(l1a,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
    l1 = tf.nn.dropout(l1,p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1,w2,strides=[1,1,1,1],padding='SAME'))
    l2 = tf.nn.max_pool(l2a,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
    l2 = tf.nn.dropout(l2,p_keep_conv)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2,w3,strides=[1,1,1,1],padding='SAME'))
    l3 = tf.nn.max_pool(l3a,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME')
    l3 = tf.reshape(l3,[-1,w4.get_shape().as_list()[0]])
    l3 = tf.nn.dropout(l3,p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3,w4))
    l4 = tf.nn.dropout(l4,p_keep_hid)
    
    pyx = tf.matmul(l4,w_o)
    return pyx

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

trX = trX.reshape(-1,28,28,1)
teX = teX.reshape(-1,28,28,1)
    

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder("float",[None,28,28,1])
Y = tf.placeholder("float",[None,10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10]) 


p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)


In [4]:
test_size = 256
batch_size = 128

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x,labels=Y))
train_opt = tf.train.RMSPropOptimizer(.001,.9).minimize(cost)
predict_opt = tf.argmax(py_x,1)

correct_prediction = tf.equal(tf.argmax(py_x, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
import numpy as np

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(200):
        for start,end in zip(range(0,len(trX),batch_size),range(128,len(trX)+1,batch_size)):
            sess.run(train_opt,feed_dict={X:trX[start:end],Y:trY[start:end],p_keep_conv:.8,
                                         p_keep_hidden:.5})
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        print(i,np.mean(np.argmax(teY[test_indices],axis=1)==
                       sess.run(predict_opt,feed_dict={X:teX[test_indices],
                                                      Y:teY[test_indices],
                                                      p_keep_conv:1.0,
                                                      p_keep_hidden:1.0})))
        if i%10==0:
            print(i,sess.run(accuracy,feed_dict={X:teX[test_indices],
                                                      Y:teY[test_indices],
                                                      p_keep_conv:1.0,
                                                      p_keep_hidden:1.0}))